In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://drive.google.com/file/d/1cocHYaXyYuDxEATlQPsoiQAl4vUi3kxo/view?usp=drive_link
import zipfile
import os
import pandas as pd
zip_file_path = '/content/drive/MyDrive/CV_PROCESSSED_DATA.zip'
extract_dir = '/content/extracted/'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Read the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the extraction directory
    zip_ref.extractall(extract_dir)

print("Zip file extracted successfully.")

Zip file extracted successfully.


In [ ]:
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split

# Directory paths
extracted_dir = '/content/extracted/CV_1'
image_dir = os.path.join(extracted_dir, 'processed_images')
csv_file_path = os.path.join(extracted_dir, 'train.csv')

# Resize images to 224x224
def resize_images(image_folder, target_size=(224, 224)):
    for filename in os.listdir(image_folder):
        if filename.endswith('.png'):
            img_path = os.path.join(image_folder, filename)
            img = Image.open(img_path)
            img = img.resize(target_size, Image.ANTIALIAS)
            img.save(img_path)

# Resize images
resize_images(image_dir)

# Load CSV file
df = pd.read_csv(csv_file_path)

# Define class labels
class_labels = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR'
}

# Split dataset into train, test, and validation
train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, val_df = train_test_split(test_val_df, test_size=0.33, random_state=42)

# Define directories for train, test, and validation sets
train_dir = '/content/train'
test_dir = '/content/test'
val_dir = '/content/validation'

# Create directories if not exists
for directory in [train_dir, test_dir, val_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Move images to respective directories
def move_images(df, source_dir, destination_dir):
    for index, row in df.iterrows():
        filename = row['id_code'] + '.png'
        source_path = os.path.join(source_dir, filename)
        if os.path.exists(source_path):
            destination_path = os.path.join(destination_dir, class_labels[row['diagnosis']], filename)
            os.makedirs(os.path.dirname(destination_path), exist_ok=True)
            os.rename(source_path, destination_path)
        else:
            print(f"Warning: File {filename} not found in {source_dir}")

move_images(train_df, image_dir, train_dir)
move_images(test_df, image_dir, test_dir)
move_images(val_df, image_dir, val_dir)

print("Dataset split into train, test, and validation sets successfully.")


Dataset split into train, test, and validation sets successfully.


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50

def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

# def create_model(input_shape, n_out):
#     input_tensor = Input(shape=input_shape)
#     base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
#     base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.5)(x)
#     x = Dense(2048, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     final_output = Dense(n_out, activation='softmax', name='final_output')(x)
#     model = Model(input_tensor, final_output)
#     return model

model = create_model(input_shape=(224, 224, 3), n_out=5)

for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True

model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                         

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5
EPOCHS = 20

# Data generator with rescaling and data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Generator for validation data (no augmentation)
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Calculate class weights to handle class imbalance
class_labels = np.unique(train_generator.classes)
class_weights = compute_class_weight('balanced', class_labels, train_generator.classes)

# Convert class weights to a dictionary
class_weight_dict = dict(enumerate(class_weights))

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weight_dict
)

# Evaluate the model on test data if available
if test_dir:
    test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
    print("Test Accuracy:", test_acc)

Found 2563 images belonging to 5 classes.
Found 363 images belonging to 5 classes.


TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [ ]:
# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5
EPOCHS = 10
LEARNING_RATE = 0.0001

# ImageDataGenerator for data augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print("Test Accuracy:", test_acc)


Found 2197 images belonging to 5 classes.
Found 732 images belonging to 5 classes.
Found 733 images belonging to 5 classes.
Epoch 1/10
69/69 [==============================] - 54s 660ms/step - loss: 1.3116 - accuracy: 0.4888 - val_loss: 1.2724 - val_accuracy: 0.4884
Epoch 2/10
69/69 [==============================] - 37s 529ms/step - loss: 1.2727 - accuracy: 0.4989 - val_loss: 1.2612 - val_accuracy: 0.4884
Epoch 3/10
69/69 [==============================] - 36s 519ms/step - loss: 1.2517 - accuracy: 0.5020 - val_loss: 1.2689 - val_accuracy: 0.4884
Epoch 4/10
69/69 [==============================] - 37s 540ms/step - loss: 1.2477 - accuracy: 0.5084 - val_loss: 1.2689 - val_accuracy: 0.4884
Epoch 5/10
69/69 [==============================] - 39s 572ms/step - loss: 1.2292 - accuracy: 0.5157 - val_loss: 1.2162 - val_accuracy: 0.4857
Epoch 6/10
69/69 [==============================] - 37s 538ms/step - loss: 1.2239 - accuracy: 0.5148 - val_loss: 1.2471 - val_accuracy: 0.4857
Epoch 7/10
69/69 [

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Clear previously defined Keras models and variables
K.clear_session()

# Optionally remove any previously defined variables or data generators
del model
del train_generator
del val_generator
del test_generator


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50

def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

# def create_model(input_shape, n_out):
#     input_tensor = Input(shape=input_shape)
#     base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
#     base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.5)(x)
#     x = Dense(2048, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     final_output = Dense(n_out, activation='softmax', name='final_output')(x)
#     model = Model(input_tensor, final_output)
#     return model

model = create_model(input_shape=(224, 224, 3), n_out=5)

for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5
EPOCHS = 10
LEARNING_RATE = 0.0001

In [ ]:
train_dir = '/content/train'
test_dir = '/content/test'
val_dir = '/content/validation'

# ImageDataGenerator for data augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 2563 images belonging to 5 classes.
Found 736 images belonging to 5 classes.
Found 363 images belonging to 5 classes.


In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = val_generator.n//val_generator.batch_size
print(STEP_SIZE_TRAIN,STEP_SIZE_VALID)

80 11


In [ ]:




















































































































# Define constants and compile the model
WARMUP_LEARNING_RATE = 0.001
WARMUP_EPOCHS = 5
LEARNING_RATE = 0.0001
EPOCHS = 20
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

model.compile(optimizer=Adam(lr=WARMUP_LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with warm-up
history_warmup = model.fit_generator(generator=train_generator,
                                     steps_per_epoch=STEP_SIZE_TRAIN,
                                     validation_data=val_generator,
                                     validation_steps=STEP_SIZE_VALID,
                                     epochs=WARMUP_EPOCHS,
                                     verbose=1)

<ipython-input-13-973aee93cd86>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_warmup = model.fit_generator(generator=train_generator,


Epoch 1/5
80/80 [==============================] - 662s 8s/step - loss: 1.6797 - accuracy: 0.4172 - val_loss: 1.2822 - val_accuracy: 0.5170
Epoch 2/5
80/80 [==============================] - 651s 8s/step - loss: 1.4640 - accuracy: 0.4397 - val_loss: 1.2972 - val_accuracy: 0.5284
Epoch 3/5
80/80 [==============================] - 629s 8s/step - loss: 1.3628 - accuracy: 0.4635 - val_loss: 1.2788 - val_accuracy: 0.4858
Epoch 4/5
80/80 [==============================] - 627s 8s/step - loss: 1.3310 - accuracy: 0.4745 - val_loss: 1.2713 - val_accuracy: 0.4347
Epoch 5/5
80/80 [==============================] - 624s 8s/step - loss: 1.3071 - accuracy: 0.4733 - val_loss: 1.2703 - val_accuracy: 0.4744


In [ ]:


# Unfreeze layers for fine-tuning
for layer in model.layers:
    layer.trainable = True

es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=3e-4, verbose=1)

callback_list = [es, rlrop]
optimizer = Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

model.summary()

# Train the model with fine-tuning
history_finetunning = model.fit_generator(generator=train_generator,
                                          steps_per_epoch=STEP_SIZE_TRAIN,
                                          validation_data=val_generator,
                                          validation_steps=STEP_SIZE_VALID,
                                          epochs=EPOCHS,
                                          callbacks=callback_list,
                                          verbose=1)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

<ipython-input-14-719d5afa12f2>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_finetunning = model.fit_generator(generator=train_generator,


Epoch 1/20
80/80 [==============================] - 2010s 25s/step - loss: 0.3384 - accuracy: 0.6906 - val_loss: 2082.4397 - val_accuracy: 0.4744 - lr: 0.0010
Epoch 2/20
80/80 [==============================] - 1998s 25s/step - loss: 0.2461 - accuracy: 0.7341 - val_loss: 0.4589 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 3/20
80/80 [==============================] - 1978s 25s/step - loss: 0.2203 - accuracy: 0.7554 - val_loss: 0.7709 - val_accuracy: 0.4659 - lr: 0.0010
Epoch 4/20
80/80 [==============================] - 1975s 25s/step - loss: 0.2339 - accuracy: 0.7396 - val_loss: 3.4511 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 5/20
80/80 [==============================] - ETA: 0s - loss: 0.2709 - accuracy: 0.7195 
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
80/80 [==============================] - 2010s 25s/step - loss: 0.2709 - accuracy: 0.7195 - val_loss: 0.7139 - val_accuracy: 0.4744 - lr: 0.0010
Epoch 6/20
80/80 [==============================] - 1968s

In [ ]:
# Splitting the dataset into 'No_DR' and 'DR' classes
df['binary_label'] = df['diagnosis'].apply(lambda x: 0 if x == 0 else 1)

# Splitting the dataset into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Define paths for train, validation, and test images
train_dir = '/content/train'
val_dir = '/content/validation'
test_dir = '/content/test'

# Create directories if not already created
for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Move images to respective directories based on the binary label
def move_images(df, source_dir, destination_dir):
    for index, row in df.iterrows():
        filename = row['id_code'] + '.png'
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, str(row['binary_label']), filename)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        os.rename(source_path, destination_path)

move_images(train_df, image_dir, train_dir)
move_images(val_df, image_dir, val_dir)
move_images(test_df, image_dir, test_dir)

# Data generators with image augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Create and compile the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print("Test Accuracy:", test_acc)